In [16]:
from dataclasses import dataclass
from pathlib import Path
import os

In [17]:
%pwd

'/Users/saiprakashlikky/Desktop/Projects/ML_projects/Flight_Fare_estimator_Project'

In [7]:
os.chdir("../")

In [8]:
%pwd

'/Users/saiprakashlikky/Desktop/Projects/ML_projects/Flight_Fare_estimator_Project'

In [18]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DatapreprocessConfig:
    root_dir: Path
    datapath: Path

In [19]:
from src.Flight_Fare_estimator_Project.constants import *
from src.Flight_Fare_estimator_Project.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_datapreprocessing_config(self) -> DatapreprocessConfig:
        config = self.config.data_preprocessing

        create_directories([config.root_dir])

        datapreprocess_config = DatapreprocessConfig(
            root_dir=config.root_dir,
            datapath=config.datapath
        )

        return datapreprocess_config


In [23]:
import warnings
import os
import urllib.request as request
import zipfile
from src.Flight_Fare_estimator_Project import logger
from src.Flight_Fare_estimator_Project.utils.common import get_size
import pandas as pd

In [24]:

class DataPreprocessing:
    def __init__(self, config):
        self.config = config

    def read_data_frame(self):
        """
        Method: read_data_frame
        Description: This method is used to load the dataset into a pandas DataFrame.
        Parameters: None
        Return: DataFrame after loading the dataset
        Version: 1.0
        """
        try:
            logger.info("Loading the dataset into a pandas DataFrame")
            # Redirect warnings to the logging system
            warnings.filterwarnings("ignore", category=UserWarning, module="pandas")
            self.df = pd.read_excel(self.config.datapath)
            logger.info("Raw data got loaded into the DataFrame")
            logger.info(f"DataFrame shape: {self.df.shape}")  # Log the shape
            return self.df

        except Exception as e:
            raise e

    def change_data_type(self, df, columns, change_type):
        """
        Method: change_data_type
        Description: This method is used to change the data type of specified columns in a DataFrame.
        Parameters:
            - df: DataFrame
            - columns: List of columns to change data type
            - change_type: Desired data type ('int', 'float', 'object', 'datetime')
        Return: DataFrame with updated data types
        Version: 1.0
        """
        try:
            logger.info("Changing data types of specified columns")
            for column in columns:
                if change_type == 'int':
                    df[column] = df[column].astype(int)
                elif change_type == 'float':
                    df[column] = df[column].astype(float)
                elif change_type == 'object':
                    df[column] = df[column].astype(object)
                elif change_type == 'datetime':
                    df[column] = pd.to_datetime(df[column], errors='coerce')

            logger.info("Data types changed successfully")
            return df

        except Exception as e:
            raise e

    def extract_month(self, df, cols):
        """
        Method: extracting month
        Description: This method is used to extract the month from the DataFrame
        Parameters:
            - df: DataFrame
            - cols: List of columns to extract the month
        Return: DataFrame with updated extracted months
        Version: 1.0
        """
        try:
            logger.info("Extracting months from the DataFrame")
            for col in cols:
                new_col_name = "Month_of_" + col
                if new_col_name not in df.columns:
                    df[new_col_name] = df["Date_of_Journey"].dt.month
            logger.info(f"Month extracted from the column {cols}")
            return df
        except Exception as e:
            raise e
    
    def extract_day(self, df, cols):
        """
        Method: extracting day
        Description: This method is used to extract the day from the DataFrame
        Parameters:
            - df: DataFrame
            - cols: List of columns to extract the month
        Return: DataFrame with updated extracted months
        Version: 1.0
        """
        try:
            logger.info("Extracting day from the DataFrame")
            for col in cols:
                new_col_name = "day_of_" + col
                if new_col_name not in df.columns:
                    df[new_col_name] = df["Date_of_Journey"].dt.day
            logger.info(f"day extracted from the column {cols}")
            return df
        except Exception as e:
            raise e


    def extract_year(self, df, cols):
        """
        Method: extracting year
        Description: This method is used to extract the year from the DataFrame
        Parameters:
            - df: DataFrame
            - cols: List of columns to extract the year
        Return: DataFrame with updated extracted years
        Version: 1.0
        """
        try:
            logger.info("Extracting years from the DataFrame")
            for col in cols:
                new_col_name = "Year_of_" + col
                if new_col_name not in df.columns:
                    df[new_col_name] = df["Date_of_Journey"].dt.year
            logger.info(f"Year extracted from the column {cols}")
            return df
        except Exception as e:
            raise e
    
    def extract_hour(self, df, cols):
        """
        Method: extracting hour
        Description: This method is used to extract the hour from the DataFrame
        Parameters:
            - df: DataFrame
            - cols: List of columns to extract the hour
        Return: DataFrame with updated extracted hours
        Version: 1.0
        """
        try:
            logger.info("Extracting hours from the DataFrame")
            for col in cols:
                new_col_name = "Hour_of_" + col
                if new_col_name not in df.columns:
                    df[new_col_name] = df[col].dt.hour
            logger.info(f"Hour extracted from the column {cols}")
            return df
        except Exception as e:
            raise e

    def extract_minute(self, df, cols):
        """
        Method: extracting minute
        Description: This method is used to extract the minute from the DataFrame
        Parameters:
            - df: DataFrame
            - cols: List of columns to extract the minute
        Return: DataFrame with updated extracted minutes
        Version: 1.0
        """
        try:
            logger.info("Extracting minutes from the DataFrame")
            for col in cols:
                new_col_name = "Minute_of_" + col
                if new_col_name not in df.columns:
                    df[new_col_name] = df[col].dt.minute
            logger.info(f"Minute extracted from the column {cols}")
            return df
        except Exception as e:
            raise e
        
    @staticmethod
    def duration_to_minutes(duration):
        parts = duration.strip().split()
        hours = 0
        minutes = 0
        for part in parts:
            if 'h' in part:
                hours = int(part.replace('h', ''))
            elif 'm' in part:
                minutes = int(part.replace('m', ''))
        return hours * 60 + minutes
    
    def extract_duration_minutes(self, df, duration_cols):
        try:
            logger.info("Converting duration from hours to minutes")
            for col in duration_cols:
                new_col_name = col + "_minutes"
                if new_col_name not in df.columns:
                    df[new_col_name] = df[col].apply(self.duration_to_minutes)
            logger.info(f"Duration converted to minutes for columns {duration_cols}")
            return df
        except Exception as e:
            raise e

    def drop_columns(self, df, cols):
        try:
            logger.info("Dropping the columns which are not required from the dataframe")
            
            # Check if columns exist before dropping
            existing_cols = set(df.columns)
            cols_to_drop = [col for col in cols if col in existing_cols]

            if cols_to_drop:
                df.drop(cols_to_drop, axis=1, inplace=True)
                logger.info(f"Dropped the {cols_to_drop} from the DataFrame The remaing Total columns are \n {df.columns}")
            else:
                logger.warning(f"Columns {cols} not found in the DataFrame. No columns dropped.")
            logger.info("Columns dropping method completed")
            return df
                
        except Exception as e:
            raise e
        
    def save_to_csv(self, df,file_path):
        """
        Method: save_to_csv
        Description: This method is used to save the DataFrame to a CSV file.
        Parameters:
            - df: DataFrame
            - file_path: Path to save the CSV file
        Return: None
        Version: 1.0
        """
        try:
            output_filepath = os.path.join(self.config.root_dir, file_path)
            if not os.path.exists(output_filepath):
                logger.info(f"Saving DataFrame to CSV file: {output_filepath}")
                df.to_csv(output_filepath, index=False)
                logger.info("DataFrame saved successfully.")
            else:
                logger.warning(f"CSV file already exists at {output_filepath}. Not saving.")
        except Exception as e:
            raise e





In [27]:
try:
    date_columns = ["Date_of_Journey", "Arrival_Time", "Dep_Time"]
    config = ConfigurationManager()
    data_preprocessing_config = config.get_datapreprocessing_config()
    data_preprocessing = DataPreprocessing(config=data_preprocessing_config)
    data_frame = data_preprocessing.read_data_frame()
    data_frame = data_preprocessing.change_data_type(data_frame, date_columns, 'datetime')
    data_frame = data_preprocessing.extract_month(data_frame, ['Month_of_journey'])
    data_frame = data_preprocessing.extract_day(data_frame, ['date_of_journey'])
    data_frame = data_preprocessing.extract_hour(data_frame, ['Arrival_Time','Dep_Time'])
    data_frame = data_preprocessing.extract_minute(data_frame, ['Arrival_Time','Dep_Time'])
    data_frame = data_preprocessing.extract_duration_minutes(data_frame,['Duration'])
    data_frame = data_preprocessing.drop_columns(data_frame,['Duration','Date_of_Journey','Dep_Time','Arrival_Time','Route','Additional_Info','Hour_of_Arrival_Time','Hour_of_Dep_Time','Minute_of_Arrival_Time','Minute_of_Dep_Time'])
    data_preprocessing.save_to_csv(data_frame, 'output.csv')  



except Exception as e:
    raise e


[2023-11-27 13:22:58,883:INFO:common:yaml file: config/config.yaml loaded successfully]
[2023-11-27 13:22:58,885:INFO:common:yaml file: params.yaml loaded successfully]
[2023-11-27 13:22:58,886:INFO:common:yaml file: schema.yaml loaded successfully]
[2023-11-27 13:22:58,887:INFO:common:created directory at: artifacts]
[2023-11-27 13:22:58,887:INFO:common:created directory at: artifacts/data_preprocessing]
[2023-11-27 13:22:58,887:INFO:446492604:Loading the dataset into a pandas DataFrame]
[2023-11-27 13:22:59,431:INFO:446492604:Raw data got loaded into the DataFrame]
[2023-11-27 13:22:59,431:INFO:446492604:DataFrame shape: (10683, 11)]
[2023-11-27 13:22:59,432:INFO:446492604:Changing data types of specified columns]
[2023-11-27 13:22:59,462:INFO:446492604:Data types changed successfully]
[2023-11-27 13:22:59,462:INFO:446492604:Extracting months from the DataFrame]
[2023-11-27 13:22:59,463:INFO:446492604:Month extracted from the column ['Month_of_journey']]
[2023-11-27 13:22:59,463:INFO

/var/folders/r4/9wpjlp4968q2ghz8t_vzp3lw0000gn/T/ipykernel_7256/446492604.py:46: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[column] = pd.to_datetime(df[column], errors='coerce')
/var/folders/r4/9wpjlp4968q2ghz8t_vzp3lw0000gn/T/ipykernel_7256/446492604.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = pd.to_datetime(df[column], errors='coerce')
/var/folders/r4/9wpjlp4968q2ghz8t_vzp3lw0000gn/T/ipykernel_7256/446492604.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = pd.to_datetime(df[column], errors='coerce')
